# Capstone: Italian Restaurants of New York City

 -------------------------------------------------------------------------

#### **Synopsis:**
#### You are a young entrepreneur with a knack for food. You have always wanted to start a niche Italian restaurant, but do not know where start the business. You have experience living in New York City, so you want to narrow it down to the neighborhoods in that city. NYC is a large city with a large market and opportunity. In this project, I explore the different factors that can affect the decision for where in New York City would be ideal to start a restaurant.

 -------------------------------------------------------------------------

#### Import Relevant Libraries

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

#### Download Dataset

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

#### Load the data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#### Clean the data to get only relevant data by selecting the 'features' attributes

In [4]:
neighborhoods_data = newyork_data['features']

#### Put the data into a dataframe by creating an empty dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#### Create loop to fill the dataframe

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head(15)    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


#### Check if there are a correct amount of rows in the dataframe

In [7]:
neighborhoods.shape

(306, 4)

#### Get the latitude and longitude values of New York City.

In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The latitude coordinate of New York City is {}.'.format(latitude))
print('The longitude coordinate of New York City is {}.'.format(longitude))

The latitude coordinate of New York City is 40.7127281.
The longitude coordinate of New York City is -74.0060152.


#### Map of the neighborhoods of New York City

In [9]:
# create map of New York using latitude and longitude values
nyc_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nyc_map)  
    
nyc_map

#### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'KYCQBXW1EBCTF5OG41QTNAXHRLXSIAEY1TWH2ZQEAM5YR10Z' #Foursquare ID
CLIENT_SECRET = 'T4PNJ2GMXFNOZ3O2LVMWHVWQMDAFBROTTOVN0OO5AE4GS4IV' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Create a function to get venues for all neighborhoods

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=100,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Insert nyc data columns into function above

In [27]:
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

#### Check the resulting dataframe

In [28]:
print(nyc_venues.shape)
nyc_venues.head()

(791, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fieldston,40.895437,-73.905643,8I06,40.895668,-73.904750,Wine Shop
1,Kingsbridge,40.881687,-73.902818,Garden Gourmet Market,40.881350,-73.903389,Gourmet Shop
2,Kingsbridge,40.881687,-73.902818,MyUnique,40.881966,-73.903584,Thrift / Vintage Store
3,Kingsbridge,40.881687,-73.902818,Mattress Firm,40.881641,-73.903061,Mattress Store
4,Kingsbridge,40.881687,-73.902818,Stop & Shop,40.882048,-73.902111,Supermarket


#### Find the number of venues were returned for each neighborhood

In [29]:
nyc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,3,3,3,3,3,3
Arlington,1,1,1,1,1,1
Astoria,1,1,1,1,1,1
Astoria Heights,1,1,1,1,1,1
Bath Beach,1,1,1,1,1,1
Battery Park City,5,5,5,5,5,5
Bay Ridge,12,12,12,12,12,12
Bay Terrace,3,3,3,3,3,3
Bayswater,2,2,2,2,2,2


#### Number of unique categories

In [30]:
print('There are {} uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))

There are 206 uniques categories.


<a id='item3'></a>

#### Use one-hot encoding to analyze the venue categories in each neighborhood

In [55]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood']    

# move 'neighborhood' column to front of dataframe
fixed_columns = [nyc_onehot.columns[-70]] + list(nyc_onehot.columns[:-70])
nyc_onehot = nyc_onehot[fixed_columns]


nyc_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,High School,Himalayan Restaurant,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon
0,Fieldston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
nyc_onehot.shape

(791, 137)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [58]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,High School,Himalayan Restaurant,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon
0,Allerton,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.333333,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Arlington,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,

#### Just show the frequency of Italian restaurants 

In [106]:
nyc_grouped = nyc_grouped[["Neighborhood","Italian Restaurant"]]
nyc_grouped

,Neighborhood,Italian Restaurant
0,Allerton,0.000000
1,Arlington,0.000000
2,Astoria,0.000000
3,Astoria Heights,0.000000
4,Bath Beach,0.000000
5,Battery Park City,0.000000
6,Bay Ridge,0.000000
7,Bay Terrace,0.000000
8,Bayswater,0.000000
9,Bedford Park,0.000000


#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [110]:
# set number of clusters
kclusters = 5

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster labels as well as the top 10 italian restaurants for each neighborhood.

In [132]:
nyc_merged = nyc_grouped.copy()

# add clustering labels
nyc_merged["Cluster Labels"] = kmeans.labels_
nyc_merged.head()

,Neighborhood,Italian Restaurant,Cluster Labels
0,Allerton,0.0,0
1,Arlington,0.0,0
2,Astoria,0.0,0
3,Astoria Heights,0.0,0
4,Bath Beach,0.0,0


In [133]:
#join dataframes to get latitude and longitude
nyc_merged = nyc_merged.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')
#nyc_merged.drop(columns = ['Borough'])
nyc_merged.head() 

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.0,0,Bronx,40.865788,-73.859319
1,Arlington,0.0,0,Staten Island,40.635325,-74.165104
2,Astoria,0.0,0,Queens,40.768509,-73.915654
3,Astoria Heights,0.0,0,Queens,40.770317,-73.894680
4,Bath Beach,0.0,0,Brooklyn,40.599519,-73.998752


#### Visualize the resulting clusters

In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

#### Examine Clusters to find the neighborhoods with a large amount of Italian restaurants

#### Cluster 1

In [137]:
df1 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 0]
df1

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.0,0,Bronx,40.865788,-73.859319
1,Arlington,0.0,0,Staten Island,40.635325,-74.165104
2,Astoria,0.0,0,Queens,40.768509,-73.915654
3,Astoria Heights,0.0,0,Queens,40.770317,-73.894680
4,Bath Beach,0.0,0,Brooklyn,40.599519,-73.998752
5,Battery Park City,0.0,0,Manhattan,40.711932,-74.016869
6,Bay Ridge,0.0,0,Brooklyn,40.625801,-74.030621
7,Bay Terrace,0.0,0,Queens,40.782843,-73.776802
7,Bay Terrace,0.0,0,Staten Island,40.553988,-74.139166
8,Bayswater,0.0,0,Queens,40.611322,-73.765968


#### Cluster 2

In [138]:
df2 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 1]
df2

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
26,Charleston,0.200000,1,Staten Island,40.530531,-74.232158
37,Concord,0.200000,1,Staten Island,40.604473,-74.084024
83,Howard Beach,0.166667,1,Queens,40.654225,-73.838138


#### Cluster 3

In [139]:
df3 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 2]
df3

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
35,Clinton Hill,0.100000,2,Brooklyn,40.693229,-73.967843
75,Greenpoint,0.125000,2,Brooklyn,40.730201,-73.954241
115,Noho,0.100000,2,Manhattan,40.723259,-73.988434
126,Prince's Bay,0.125000,2,Staten Island,40.526264,-74.201526
169,Yorkville,0.083333,2,Manhattan,40.775930,-73.947118


#### Cluster 4

In [140]:
df4 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 3]
df4

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
22,Carnegie Hill,0.222222,3,Manhattan,40.782683,-73.953256
54,Eltingville,0.250000,3,Staten Island,40.542231,-74.164331
97,Little Italy,0.250000,3,Manhattan,40.719324,-73.997305


#### Cluster 5

In [141]:
df5 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 4]
df5

,Neighborhood,Italian Restaurant,Cluster Labels,Borough,Latitude,Longitude
43,Downtown,0.033333,4,Brooklyn,40.690844,-73.983463
86,Hunters Point,0.040000,4,Queens,40.743414,-73.953868
123,Park Slope,0.043478,4,Brooklyn,40.672321,-73.977050


#### Conclusion

Based on the 5 clusters created, Clusters 2, 3, 4, and 5 seem to be heavily populated areas with an abundance of Italian restaurants. These neighborhoods are also belong to some of the largest tourist attractions in the city, mostly located in southern Manhattan. Ideal places for an Italian restaurant may be in places in the adjacent neighborhoods of those near southern Manhattan.  